## Aufgabe 1 
## Implementierung einer Reward-Funktion für einen Reinforcement Learning (RL) Agenten

In dieser Aufgabe sollen Sie eine Reward-Funktion für einen RL-Agenten implementieren. Der Agent hat die Aufgabe, ein bestimmtes Ziel zu erreichen und dabei möglichen statischen und dynamischen Hindernissen auszuweichen.

Die Reward-Funktion ist ein zentrales Element in der Umgebung eines RL-Algorithmus. Sie bewertet die Aktionen des Agenten und liefert Rückmeldungen, die dem Agenten helfen, zu lernen und Entscheidungen zu treffen, die ihn seinem Ziel näherbringen. Ihre Aufgabe besteht darin, diese Funktion zu implementieren und dabei die Vielseitigkeit und Anpassungsfähigkeit einer solchen Funktion zu berücksichtigen.

Folgen Sie diesen Schritten:

1. **Erstellen Sie eine Reward-Funktion als Dictionary.** Jeder Eintrag in diesem Dictionary repräsentiert einen bestimmten Aspekt oder eine bestimmte Dimension der Bewertung. Beispielsweise kann ein Eintrag die Belohnung für das Erreichen des Ziels, eine Strafe für das Kollidieren mit einem Hindernis oder das Stehenbleiben des Agenten repräsentieren.

2. **Überlegen Sie sich, welche Aspekte des Verhaltens des Agenten Sie belohnen oder bestrafen möchten,** und implementieren Sie diese in Ihrer Reward-Funktion. Einige Beispiele könnten sein: das Erreichen des Ziels, das Vermeiden von Kollisionen, das Halten einer bestimmten Distanz zum Ziel oder das Bewahren einer bestimmten Orientierung zum Ziel.

3. **Nutzen Sie die Hilfsfunktionen der Robot-Klasse,** um wichtige Informationen über den Zustand des Roboters zu erhalten. Diese Funktionen können Informationen über die Position und Ausrichtung des Roboters, die Entfernung zu Hindernissen und ähnliches liefern. Überlegen Sie, wie Sie diese Informationen nutzen können, um die Belohnungen und Strafen in Ihrer Reward-Funktion zu berechnen. 

> `is_staying_in_place(robot.last_positions)` Diese Funktion prüft, ob der Roboter in den letzten Positionen stillgestanden hat. Das ist wichtig zu wissen, da es Anzeichen dafür sein könnte, dass der Roboter in einem Hindernis feststeckt oder in einer Schleife feststeckt, in der er immer wieder die gleiche Aktion ausführt.

> `robot.initialGoalDist` Diese Variable speichert die ursprüngliche Entfernung des Roboters zum Ziel, als der Lernprozess begonnen hat. Dieser Wert kann dazu genutzt werden, um zu überprüfen, ob der Roboter näher am Ziel ist als zu Beginn, was ein gutes Zeichen für Fortschritt wäre.

> `currentLinVel = np.around(robot.state_raw[robot.time_steps - 1][4], decimals=5)` Dieser Ausdruck berechnet die aktuelle lineare Geschwindigkeit des Roboters. robot.state_raw[robot.time_steps - 1][4] greift auf den neuesten Status des Roboters zu und holt die lineare Geschwindigkeit, die dann auf 5 Dezimalstellen gerundet wird.

> `lastLinVel = np.around(robot.state_raw[robot.time_steps - 2][4], decimals=5)` Ähnlich wie der vorherige Ausdruck berechnet dieser die lineare Geschwindigkeit des Roboters im vorherigen Zeitschritt. Der einzige Unterschied ist, dass hier robot.time_steps - 2 verwendet wird, um auf den vorherigen Status des Roboters zuzugreifen.

> `currentAngVel = np.around(robot.state_raw[robot.time_steps - 1][5], decimals=5)` Dieser Ausdruck berechnet die aktuelle Winkelgeschwindigkeit des Roboters, ähnlich wie die Berechnung der linearen Geschwindigkeit.

> `lastAngVel = np.around(robot.state_raw[robot.time_steps - 2][5], decimals=5)` Wie der vorherige Ausdruck berechnet dieser die Winkelgeschwindigkeit des Roboters im vorherigen Zeitschritt.

Das bereitgestellte Grundgerüst soll Ihnen dabei helfen, diese Aufgabe zu bewältigen. Denken Sie daran, dass Ihre Reward-Funktion flexibel und anpassungsfähig sein sollte, um auf die spezifischen Anforderungen des RL-Problems eingehen zu können.

In [1]:
import numpy as np
from utils import is_staying_in_place

def createReward(robot, dist_new, dist_old, reachedPickup, collision, runOutOfTime):
    """
    Creates a (sparse) reward based on the euklidian distance, if the robot has reached his goal and if the robot
    collided with a wall or another robot.

    :param robot: robot
    :param dist_new: the new distance to goal (after the action has been taken)
    :param dist_old: the old distance to goal (before the action has been taken)
    :param reachedPickup: True if the robot reached his goal in this step
    :param collision: True if the robot collided with a wall or another robot
    :param runOutOfTime: True if the maximum steps were taken by the robot and its still alive
    :return: returns the result of the fitness function
    """

    reward = {}
    reward['nothing'] = 0
    reward['anothernothing'] = 0
    
    # Ihr könnt beliebig viele Dictionary Einträge für die unterschiedlichen Aspekte der
    # Reward Funktion erstellen. In tensorboard(weiter unten wird es beschrieben) werden diese
    # dann unter dem Namen angezeigt

    return reward

## Return, Advantage und GAE

In der Welt des Reinforcement Learning (RL) verwenden wir oft Begriffe wie "Vorteil" (Advantage) und "Rückgabe" (Return) zur Beschreibung bestimmter Konzepte.

1. **Return:** Im Kontext von RL bezieht sich Return auf die gesamte zukünftige Belohnung, die ein Agent von einem bestimmten Zustand aus erwartet. Es wird oft als Diskontierte Summe zukünftiger Belohnungen bezeichnet. Hierbei wird eine Diskontierungsrate (oft Gamma genannt) verwendet, die die Wichtigkeit zukünftiger Belohnungen im Vergleich zur aktuellen Belohnung verringert. Formell kann der Return als `Gt = Rt+1 + γRt+2 + γ^2Rt+3 + ...` berechnet werden, wobei `Rt+1, Rt+2, ...` zukünftige Belohnungen sind und γ die Diskontierungsrate ist.

2. **Advantage:** Advantage ist eine Methode zur Quantifizierung, wie viel besser eine bestimmte Aktion in einem bestimmten Zustand ist, im Vergleich zu dem, was wir im Durchschnitt von diesem Zustand erwarten würden. Es misst im Grunde genommen den Unterschied zwischen der Q-Funktion und der V-Funktion in RL. Die Q-Funktion gibt den erwarteten Return für ein gegebenes Zustands-Aktions-Paar an, während die V-Funktion den erwarteten Return für einen gegebenen Zustand unabhängig von der spezifischen Aktion angibt. Die Advantage-Funktion ist dann definiert als `A(s, a) = Q(s, a) - V(s)`.

Die Berechnung von Advantage und Return ist zentral für viele RL-Algorithmen, da sie hilft zu bestimmen, welche Aktionen besser als andere sind und welche Zustände wertvoller sind. Sie werden oft verwendet, um die Richtung zu bestimmen, in die sich die Parameter unseres Modells bewegen sollten, um das Lernen zu verbessern.

Einige Algorithmen, wie der Proximal Policy Optimization (PPO) Algorithmus, verwenden eine Technik namens Generalized Advantage Estimation (GAE) zur Berechnung der Advantage. GAE ist eine Methode zur Verwendung von gewichteten Summen mehrerer Returns, um die Schätzung zu verbessern. Es hat zwei Parameter, die Diskontierungsrate γ und einen weiteren Parameter λ, der bestimmt, wie die Gewichtung zwischen verschiedenen Returns vorgenommen wird. In Ihrer Funktion get_advantages wird GAE verwendet, um den Advantage zu berechnen, der dann zur Verbesserung der Policy und Value-Funktionen des Modells verwendet wird.

## Aufgabe 2

## Implementierung der GAE Funktion für Proximal Policy Optimization
In dieser Aufgabe sollen Sie die Berechnung von Advantages und Returns für den Proximal Policy Optimization (PPO) Algorithmus implementieren. Dabei wird die Technik der Generalized Advantage Estimation (GAE) verwendet, um die Advantages zu berechnen.

#### masks
Die Variable masks wird verwendet, um das Ende einer Episode zu kennzeichnen. In der Welt des Reinforcement Learning ist eine Episode eine Reihe von Interaktionen zwischen dem Agenten und der Umgebung, die mit dem Erreichen eines Endzustands endet.

Wenn Sie eine Umgebung haben, in der Episoden enden können (wie zum Beispiel in dieser Simulation, die endet, wenn der Agent ein bestimmtes Ziel erreicht, ein bestimmter Zeitraum abgelaufen ist oder der Agent gecrasht ist), dann verwenden Sie oft eine Maske, um zu kennzeichnen, wo diese Endpunkte sind.

Diese Masken werden später im Code verwendet, um sicherzustellen, dass der zukünftige Return korrekt berechnet wird, wenn eine Episode endet. Wenn eine Episode endet, gibt es keine zukünftige Belohnung mehr zu berücksichtigen, so dass die Diskontierte Summe zukünftiger Belohnungen ab diesem Punkt aufhören sollte. Die `masks`-Variable hilft dabei, diese Punkte zu kennzeichnen, so dass die zukünftigen Belohnungen korrekt berechnet werden können.

In [2]:
def get_advantages(gamma, _lambda, values, masks, rewards):
    """
    Computes the advantages of the given rewards and values.

    :param values: The values of the states.
    :param masks: The masks of the states.
    :param rewards: The rewards of the states.
    :return: The advantages of the states.
    """
    advantages = []
    returns = []
    gae = 0

    for i in reversed(range(len(rewards))):
        # TODO: Berechnen Sie den Temporal Difference Fehler 'delta'
        delta = None

        # TODO: Berechnen Sie die Generalized Advantage Estimate 'gae'
        gae = None

        # TODO: Berechnen Sie den Return und fügen Sie ihn zur Liste der Returns hinzu
        return_ = None
        returns.insert(0, return_)

    advantages = torch.FloatTensor(advantages).to(device)
    norm_adv = (advantages - advantages.mean()) / (advantages.std() + 1e-10) if advantages.numel() > 1 else advantages

    returns = torch.FloatTensor(returns).to(device)

    return norm_adv, returns

## Aufgabe 3
## Erstellen eines Eingabemoduls für ein Reinforcement Learning (RL) neuronales Netz

In dieser Aufgabe sollen Sie ein Eingabemodul für ein neuronales Netz in PyTorch implementieren, das mehrere unterschiedliche Eingaben verarbeitet und zu einer gemeinsamen Repräsentation zusammenführt. Dieses Eingabemodul ist Teil einer größeren Architektur, die später im Rahmen von Reinforcement Learning (z. B. PPO) trainiert wird.

Das Netz erhält verschiedene Eingangsdaten über den Zustand eines Roboters, darunter:
- Lidar-Scans über mehrere Zeitschritte (z. B. 4 Zeitschritte)
- Orientierung des Roboters zum Ziel
- Entfernung zum Ziel
- Geschwindigkeit des Roboters (lineare und rotatorische Komponenten)

Ziel ist es, diese Datenquellen getrennt mit passenden Schichten (z. B. Convolutional Layer für die Lidar-Daten und Dense Layer für die anderen Inputs) zu verarbeiten, ihre Features zu extrahieren und am Ende zu einem flachen, konsolidierten Vektor zu kombinieren. Dieser Vektor kann anschließend von weiteren Netzwerkschichten (z. B. Policy- und Value-Netzwerk) genutzt werden. Diese Netze müssen Sie nicht erstellen.

## Vorgehen
1. Erstellen Sie eine eigene Klasse, die von `nn.Module` erbt (z. B. BigInput).
2. Verarbeiten Sie jede Eingabekomponente getrennt, z. B. mithilfe von Convolutional- oder Linear-Schichten. Diese Einzelschritte sind nicht vorgeschrieben, sollten aber dem Typ der Eingabedaten angemessen sein.
3. Kombinieren Sie die resultierenden Feature-Vektoren zu einem gemeinsamen Eingabevektor.
4. Implementieren Sie die Methode forward(self, ...), die die Eingaben entgegennimmt, verarbeitet und die finale Feature-Repräsentation zurückgibt. Diese Methode definiert den Ablauf der Vorwärtsauswertung des Netzwerks.
5. Der Rückgabewert der forward-Methode muss ein Tensor der Form (Batchgröße, `self.out_features`) sein.
6. Der Wert von self.out_features gibt die Anzahl der Ausgabeneuronen des Moduls an und muss zwingend gesetzt werden. Andere Module im Netzwerk werden sich auf diese festgelegte Ausgabegröße verlassen.
7. Nutzen Sie die Hilfsfunktion `initialize_hidden_weights`, um die Gewichte der Layer initial zu setzen (diese Funktion wird vorab bereitgestellt).

Sie dürfen frei entscheiden, wie viele Schichten, Aktivierungsfunktionen oder Verarbeitungsschritte Sie zwischen Eingabe und Ausgabe verwenden – wichtig ist, dass die Ausgabeform korrekt ist und alle Eingabedaten sinnvoll verarbeitet werden.

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from utils import initialize_hidden_weights

class BigInput(nn.Module):
    def __init__(self, scan_size):
        """
        Eingabemodul für verschiedene Roboterzustände im Reinforcement Learning.

        :param scan_size: Anzahl der Lidar-Werte pro Zeitschritt.
        """
        super(BigInput, self).__init__()

        self.time_steps = 4  # Anzahl vergangener Zeitschritte
        self.out_features = 32  # Muss gesetzt werden – Ausgabegröße des Moduls

        # Beispielhafte Layer (Platzhalter – durch eigene Verarbeitung ersetzen)
        self.lidar_layer = nn.Identity()
        self.orientation_layer = nn.Identity()
        self.distance_layer = nn.Identity()
        self.velocity_layer = nn.Identity()
        self.final_layer = nn.Identity()

    def forward(self, lidar, orientation, distance, velocity):
        """
        Verarbeitet alle Eingaben und gibt einen zusammengeführten Feature-Vektor aus.

        :param lidar: Tensor mit Lidar-Daten (z. B. [B, T, S])
        :param orientation: Tensor mit Zielorientierung (z. B. [B, T, 2])
        :param distance: Tensor mit Distanz zum Ziel (z. B. [B, T, 1])
        :param velocity: Tensor mit Geschwindigkeit (z. B. [B, T, 2])
        :return: Tensor der Form (Batchgröße, self.out_features)
        """
        lidar_feat = self.lidar_layer(lidar)
        orientation_feat = self.orientation_layer(orientation)
        distance_feat = self.distance_layer(distance)
        velocity_feat = self.velocity_layer(velocity)

        combined = torch.cat((lidar_feat, orientation_feat, distance_feat, velocity_feat), dim=1)
        output = self.final_layer(combined)
        return output

    def get_in_features(self, h_in, layers_dict):
        """
        Berechnet die Ausgabelänge eines 1D-Convolutional Layers nach mehreren Verarbeitungsschritten.

        Diese Funktion ist nützlich, wenn Sie mehrere Conv1d-Schichten definieren und die resultierende
        Feature-Länge vor der Verwendung in einem Linear-Layer ermitteln möchten.

        :param h_in: Ausgangslänge des Eingabevektors (z. B. die Lidar-Scan-Länge)
        :param layers_dict: Liste von Dictionaries mit Layer-Parametern:
               [{'kernel_size': ..., 'stride': ..., 'padding': ..., 'dilation': ...}, ...]
        :return: berechnete Länge nach allen Convolution-Schritten
        """
        for layer in layers_dict:
            kernel_size = layer['kernel_size']
            stride = layer['stride']
            padding = layer['padding']
            dilation = layer['dilation']
            h_in = ((h_in + 2 * padding - dilation * (kernel_size - 1) - 1) / stride) + 1
        return int(h_in)

### Simulationsumgebung

Es gibt verschiedene Parameter, die Sie anpassen können, um das Training zu steuern. Die einzelnen Parameter sind in den Kommentaren erklärt. 

Nachdem Sie alle Parameter nach Ihren Bedürfnissen angepasst haben, können Sie die Funktion startSimulation aufrufen und das Training beginnen. Die ausgewählten Level-Dateien werden als Trainingsumgebungen verwendet, und die Funktionen createReward und get_advantages werden verwendet, um die Belohnungen und Vorteile zu berechnen, die für das Training des Agenten benötigt werden.

In [4]:
import PyQt5.QtGui
from startSim import startSimulation
import random

args = {}
level_files = ['ez.svg', 'ez2.svg','ez4.svg', 'Simple.svg', 'tunnel2.svg']
#level_files = ['kreuzung.svg', 'Funnel.svg', 'tunnel2.svg', 'ez3.svg'] + ['ez.svg', 'ez2.svg','ez4.svg', 'Simple.svg']# Model Parameters
random.shuffle(level_files)

# Dies ist der Pfad zum Ordner, in dem die trainierten Modelle gespeichert werden.
args['ckpt_folder']= './models/mymodel'
# Der Name des trainierten Modells, das gespeichert oder geladen werden soll.
# Beim testen schaut genau nach welches model ihr wählen wollt "model_best" "model_solved" oder "model_final"
args['model_name']= 'model_best'
# Der Modus, in dem die Simulation laufen soll. Hier ist es auf "train" gesetzt, was 
# bedeutet, dass wir ein Training durchführen. 'train' oder 'test'
args['mode']= 'test'
# 'small', 'big' or custom network 
args['inputspace'] = BigInput

# Train Parameters

# Wenn auf True gesetzt, wird das Modell von einer früheren Checkpoint-Datei wiederhergestellt 
# und das Training wird fortgesetzt. Wenn auf False gesetzt, wird das Modell von Grund auf neu trainiert.
args['restore']=False
# Die Anzahl der Schritte, die in jeder Episode durchgeführt werden sollen.
args['steps']=1000
# Die Anzahl der Schritte, die durchgeführt werden sollen, bevor trainiert wird.
args['update_experience']=2500
# Die Anzahl der Minibatches zum Aufteilen der Trainingsdaten während einer Trainingsepoche.
args['batches']=32
# Die Anzahl der Epochen, für die das Modell trainiert werden soll.
args['K_epochs']=10
# Der Diskontierungsfaktor, der verwendet wird, um zukünftige Belohnungen zu bewerten.
args['gamma']=0.99
# Der Faktor Lambda im Generalized Advantage Estimation (GAE) Algorithmus, der dazu dient, den 
# Bias-Variance-Tradeoff bei der Schätzung der Vorteile zu steuern.
args['_lambda']=0.99
# Die Lernrate, die den Schrittgrößenparameter beim Aktualisieren der Gewichte des Modells bestimmt.
args['lr']=0.0002

startSimulation(args, level_files, createReward, get_advantages)

Load checkpoint from ./models/mymodel/PPO_continuous_model_best.pth


/home/nex/Dokumente/Code/SauRoN/PPO/Algorithm.py:222: UserWarning: Could not restore model from models/mymodel/PPO_continuous_model_best.pth. Falling back to train mode.
  warnings.warn(f"Could not restore model from {path}. Falling back to train mode.")


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 1081 but got size 2 for tensor number 1 in the list.

## Visualisierung mit tensorboard

TensorBoard ist ein Visualisierungstool für Machine Learning. Es ermöglicht es den Benutzern, die verschiedenen Aspekte ihres Machine Learning-Modells zu visualisieren und zu verstehen. Dabei können die Lernkurven von Modellen, Histogramme von Gewichtungen oder Biases, Bilder oder Texte, die durch das Modell generiert werden, sowie viele andere Dinge visualisiert werden.

Im Kontext von Reinforcement Learning kann TensorBoard dazu verwendet werden, Metriken wie den durchschnittlichen Reward, die durchschnittliche Anzahl von Schritten pro Episode oder andere benutzerdefinierte Metriken zu verfolgen. Diese Metriken können im Laufe der Zeit verfolgt und visualisiert werden, um zu verstehen, wie das Modell lernt und sich verbessert (oder auch nicht).

Sie können es starten, indem Sie `tensorboard --logdir=pfad_zu_ihren_logs` in Ihrer Konsole ausführen. Hierbei ist pfad_zu_ihren_logs der Pfad zu dem Verzeichnis, in dem die Log-Dateien gespeichert sind, die von SauRoN während des Trainings erstellt wurden. SauRoN erstellt diese Dateien unter dem von Ihnen angegeben `ckpt_folder`.

![Reward](./reward_tb.png)

#### Hinweise

Reinforcement-Learning beginnt oft mit kleinen, aber ermutigenden Fortschritten. Schon nach wenigen Trainingszyklen sollten Sie in der Lage sein, erste positive Ergebnisse sowohl in der Simulation als auch in TensorBoard zu beobachten. Ein konkreter Erfolg in dieser Lernumgebung könnte beispielsweise sein, dass Ihre Agenten lernen, Kollisionen zu vermeiden oder bestimmte Wegpunkte zu erreichen.

Dennoch ist es wichtig, realistische Erwartungen zu haben: Das Erreichen einer 100%igen Erfolgsrate beim Erreichen des Ziels ist eine beachtliche Herausforderung und erfordert wahrscheinlich viel mehr Training, als man zunächst annimmt. Machen Sie sich jedoch keine Sorgen, wenn der Fortschritt nicht sofort sichtbar ist – das ist völlig normal und Teil des Lernprozesses.

Sie haben die Freiheit, das Training jederzeit zu unterbrechen und später fortzusetzen. Der aktuell beste Stand Ihres Modells wird kontinuierlich gespeichert, sodass Sie keine Fortschritte verlieren. Dies ermöglicht eine flexible Anpassung des Lernprozesses an Ihren Zeitplan.

Darüber hinaus bietet die Simulation die Möglichkeit, die aktuellen Gewichtungen zu speichern. Dies kann besonders wertvoll sein, wenn Sie bestimmte Gewichtungen für zukünftige Experimente, Analysen oder Anwendungen beibehalten möchten.

#### Tipps zum Testen und Experimentieren

**Schrittweise Anpassung der Parameter:** Beginnen Sie mit kleineren Werten für die Lernrate, das Diskontierungsfaktor Gamma und die Anzahl der Trainingsepochen. Beobachten Sie, wie sich die Leistung des Agenten ändert, wenn Sie diese Werte schrittweise erhöhen.

**Nutzen Sie TensorBoard:** TensorBoard ist ein großartiges Werkzeug, um den Lernprozess visuell zu überwachen. Verwenden Sie es, um Belohnungen, Verluste und andere Metriken im Laufe der Zeit zu verfolgen. Dies kann Ihnen helfen, zu verstehen, wie gut Ihr Agent lernt und wann Sie möglicherweise Anpassungen vornehmen müssen.

**Experimentieren Sie mit verschiedenen Umgebungen:** Die bereitgestellten Level-Dateien repräsentieren verschiedene Umgebungen, die unterschiedliche Herausforderungen für den Agenten darstellen. Versuchen Sie, Ihr Modell in verschiedenen Umgebungen zu trainieren und zu testen, um zu sehen, wie gut es sich generalisieren kann.

**Auswertung der Ergebnisse:** Vergessen Sie nicht, Ihre trainierten Modelle auch zu testen. Sie können dies tun, indem Sie den Modus auf 'test' setzen und die Simulation starten. Dies gibt Ihnen eine gute Vorstellung davon, wie gut Ihr Agent in einer unabhängigen Umgebung abschneidet.

**Training auf unterschiedlichen Umgebungen** Das Training auf unterschiedlichen Umgebungen (auch bekannt als Curriculum Learning) kann sehr nützlich sein, insbesondere in komplexen Umgebungen, in denen ein RL-Agent Schwierigkeiten haben könnte, relevante Strategien zu lernen. Durch den Start in einfacheren Umgebungen kann der Agent grundlegende Strategien lernen, die ihm in schwierigeren Umgebungen helfen.

**Frühes Stoppen des Trainings**
Frühes Stoppen ist eine Technik, die in vielen Arten von Machine Learning verwendet wird, um Overfitting zu vermeiden. Im Kontext von Reinforcement Learning kann es auch dazu verwendet werden, um unnötiges Training zu vermeiden, wenn der Agent bereits eine akzeptable Leistung erreicht hat.

Die Idee ist, das Training zu stoppen, sobald die Leistung des Agenten aufhört, sich signifikant zu verbessern, oder sogar zu beginnen, sich zu verschlechtern. In Reinforcement Learning könnte dies bedeuten, dass das Training gestoppt wird, wenn die durchschnittliche Belohnung über eine Reihe von Episoden hinweg konstant bleibt oder zu sinken beginnt.